In [21]:
import numpy as np
import pandas as pd
import math
import statistics
from statistics import mode
import pprint



        
def open_file(filename):
    if filename == 'tic-tac-toe.data':
        col_name = ['top-l','top-m','top-r','mid-l','mid-m','mid-r','bot-l','bot-m','bot-r','class']
        df = pd.read_csv(filename,names = col_name)   
    if filename == 'balance-scale.data':
        col_name = ['class','lft-weight','lft-dist','rght-weight','rght-dist']
        df = pd.read_csv(filename,names = col_name)
        #move class to the last row
        cols = df.columns.tolist()
        cols = cols[1:] + cols[:1]
        df = df[cols]
    if filename == 'agaricus-lepiota.data':
        col_name = ['class','cap-shape','cap-surface','cap-color','bruises','odor','gill-attachment','gill-spacing','gill-size',
        'gill-color','stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring', 'stalk-color-above-ring',
        'stalk-color-below-ring','veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat']
        df = pd.read_csv(filename,names = col_name)
        #move class to the last row
        cols = df.columns.tolist()
        cols = cols[1:] + cols[:1]
        df = df[cols]
    if filename == 'car.data':
        col_name  = ['Buying','Maint','Doors','Persons','Lug Boot','Safety','class']
        df = pd.read_csv(filename,names = col_name)

    if filename =='zoo.data':
        #import the dataset
        col_name = ['animal_name','hair','feathers','eggs','milk','airborne','aquatic','predator','toothed','backbone','breathes','venomous','fins','legs','tail','domestic','catsize','class']
        df = pd.read_csv(filename,names =col_name)
        #we drop this feature,since this is not good feature to split the data
        df = df.drop('animal_name',axis=1)
    #print(df)
    #split into training and testing 
    #shuffle data to randomize
    #80% training, %20 test
    #shuffle
    df = df.sample(frac=1)
    training = df.sample(frac = 0.8)
    remainder = df.drop(training.index)
    test = remainder.sample(frac = 0.2)
 
    return df,test,training

In [22]:

def entropy(df):
    #entropy(S) = summation { -p*log_2(p)}
    target_col = df.keys()[-1]
    classifiers = df[target_col].unique()
    e = 0
    for classifier in classifiers:
        #print('entropy eq:', classifier)
        frac = df[target_col].value_counts()[classifier] /len(df[target_col])
        e += -frac*math.log2(frac)
    return e

def gain(df,split_attribute_name):
    """ 
    #info gain of aattribute
    #gain(s,a) = entropy(s) - summation for v in values(A){ (|S_v|/|S|)*Entropy(S_v) }
    """
    
    target_col = df.keys()[-1]
    values = df[split_attribute_name].unique()
    ''' 
    print('classifier',classifiers)
    for i in values:
        print(i)
    print('value',values)
    '''
    s = entropy(df)
    #print('s',s)
    summation = 0
    for v in values:
        frac = df[split_attribute_name].value_counts()[v]/(len(df[split_attribute_name]))
        #print('v',v)
        sv = df[df[split_attribute_name] == v]
        #print(sv)
        entropy_sv = entropy(sv)
        #print('sv',entropy_sv)
        summation += frac *entropy_sv
    g = s - summation
    
    return g

def find_best_feature(df):
    """ 
    traverse throught each attribute and append to an array
    return the best attribute
    """
    info = []
    #print('df')
    #print(df)
    #print('keys',df.keys()[:-1])

    for key in df.keys()[:-1]:
        #print('k',key)
        info.append(gain(df,key))
    ''' 
    print('info',info)
    print('npDFS', np.argmax(info))
    print('keys',df.keys()[:-1][np.argmax(info)])
    #return the best KEY based on the value of 
    '''
    #attribute of best info
    return df.keys()[:-1][np.argmax(info)]

def decision_tree(df,og_df,features,target_attribute_name="class",parent_node_class = None):
    """
    recursively creates the tree
    data structure used: nested dicts
    """
    #If all target_values have the same value,return this value
    if len(np.unique(df[target_attribute_name])) <= 1:
        #print('have no more data')
        #print(np.unique(df[target_attribute_name])[0])
        return np.unique(df[target_attribute_name])[0]
    
    #if dataset is empty
    elif len(df) ==0:
        #print('len0',np.unique(og_df[target_attribute_name])[np.argmax(np.unique(og_df[target_attribute_name],return_counts = True)[1])])
        return np.unique(og_df[target_attribute_name])[np.argmax(np.unique(og_df[target_attribute_name],return_counts = True)[1])]
    #if feature is empty
    elif len(features) ==0:
        return parent_node_class
    #if none base conditions hold true grow tree

    else:
        parent_node_class = np.unique(df[target_attribute_name])[np.argmax(np.unique(df[target_attribute_name],return_counts=True)[1])]
        #print('pnc',parent_node_class)

        best_feature = find_best_feature(df)
            
        #Create the tree structure of best feature recieved from df
        tree = {best_feature:{}}

        #Remve the feature with the best info gain
        features = [i for i in features if i!= best_feature]

        for value in np.unique(df[best_feature]):
            value = value
            sub_data = df.where(df[best_feature]==value).dropna()
            #call recursion
            subtree = decision_tree(sub_data,og_df,features,target_attribute_name,parent_node_class)
            #add subtree
            tree[best_feature][value] = subtree

        return tree
    

    


In [23]:
def predict(query,tree,default =1):
    """
    traverse through the test
    if the key which is the  attribute is in the tree
        try accessing it
    else return 1
    if you can access it and we check its a dict
    recursivly call the function
    otherwise return what class it is 
    """
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]]
            except:
                return default

            result = tree[key][query[key]]
            if isinstance(result,dict):
                return predict(query,result)
            else:
                return result

def test(data, tree):
    """
    populates the predicted df with the prediction or 1 if incorrect
    to calc accuracy calc the ones that are not equal too 1 and div by len of data passed in and take percentage
    return lisst of predicted class
    """
    # [{column -> value}, … , {column -> value}]
    queries = data.iloc[:,:-1].to_dict(orient = 'records')
    predicted =pd.DataFrame(columns = ["predicted"])
    #print('p',predicted)
    #print('queries',queries)
    #print(data)
    #print('------------------')
    #predictions for what the index in the data is
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0)
    #print(predicted)
    p = predicted["predicted"].to_numpy()
    d = data["class"].to_numpy()
    ratio = np.sum(p==d)
    #print(ratio)
    #print(len(data))
    print("The Prediction accuracy is:",ratio/len(data)*100,'%')
    return p
    


In [24]:
def most_frequent(l):
    return mode(l)

def random_forest(df,testing):
    """
    split df into log2(len(df))
    for each subset
        make sub decsion tree
    run prediction for each subtree
    get cols from rows of nested list
    get most occured from each col
    get ratio of how many guessed correctly 
    get percentage of guessed right
    """

    #print(df)
    #print(testing["class"])
    #print(len(testing["class"]))
    splits =int(math.log2(len(df)))
    split_trainng = np.array_split(df,splits)
    """ 
    for i in split_trainng:
        print('-----')
        print(i)
        print('-------')
    """
    trees = []
    for subset in split_trainng:
        tree = decision_tree(subset,subset,subset.columns[:-1])
        trees.append(tree)
    predictions = []
    for tree in trees:
        prediction = test(testing,tree)
        predictions.append(prediction)
        """ 
        print('---------')
        pprint.pprint(i)
        print('---------')
        """
    """ 
    print('pred')
    for i in predictions:
        print('---------')
        print(i)
        print('---------')
    """
    tpredictions = list(zip(*predictions))
    guesses = pd.DataFrame(columns = ["predicted"])
    for i in range(len(tpredictions)):
        #guesses.append(most_frequent(i))
        guesses.loc[i,"predicted"] = most_frequent(tpredictions[i])
 
    """
    print('tpred')
    for i in tpredictions:
        print('---------')
        print(i)
        print('---------')
    """

    g = guesses["predicted"].to_numpy()
    t = testing["class"].to_numpy()
    ratio = np.sum(g==t)
    print("The Prediction accuracy when doing the random forest is:",ratio/len(testing)*100,'%')

    return
def main():
    #filename = 'zoo.data'
    filename = 'car.data'
    #filename = 'agaricus-lepiota.data'
    df,testing,training = open_file(filename)
    tree = decision_tree(training,training,training.columns[:-1])
    #pprint.pprint(tree)
    y = test(training,tree)
    x = test(testing,tree)
    random_forest(training,testing)
    return

main()

The Prediction accuracy is: 100.0 %
The Prediction accuracy is: 85.5072463768116 %
The Prediction accuracy is: 78.26086956521739 %
The Prediction accuracy is: 84.05797101449275 %
The Prediction accuracy is: 75.36231884057972 %
The Prediction accuracy is: 78.26086956521739 %
The Prediction accuracy is: 65.21739130434783 %
The Prediction accuracy is: 75.36231884057972 %
The Prediction accuracy is: 78.26086956521739 %
The Prediction accuracy is: 75.36231884057972 %
The Prediction accuracy is: 75.36231884057972 %
The Prediction accuracy is: 79.71014492753623 %
The Prediction accuracy when doing the random forest is: 84.05797101449275 %
